# v2.3 Check Physics

This notebook is intended to test traing and then saving a model object for later use.


## Environment Setup

In [ ]:
import numpy as np
import sys
sys.path.append('..')
import pickle
import logging
import os.path as osp
import tensorflow as tf
from moisture_rnn_pkl import pkl2train
from moisture_rnn import RNNParams, RNNData, RNN, rnn_data_wrap
from utils import hash2, read_yml, read_pkl, retrieve_url, Dict, print_dict_summary, print_first, str2time, logging_setup
from moisture_rnn import RNN
import reproducibility
from data_funcs import rmse, to_json, combine_nested, subset_by_features, build_train_dict
from moisture_models import run_augmented_kf
import copy
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import time

In [ ]:
logging_setup()

In [ ]:
filename = "fmda_rocky_202403-05_f05.pkl"
retrieve_url(
    url = f"https://demo.openwfm.org/web/data/fmda/dicts/{filename}", 
    dest_path = f"../data/{filename}")

In [ ]:
file_paths = [f'../data/{filename}']

In [ ]:
# # read/write control
# train_file='../data/train.pkl'
# train_create=True   # if false, read
# train_write=False
# train_read=False

In [ ]:
# Params used for data filtering
params_data = read_yml("../params_data.yaml") 
params_data

In [ ]:
# Params used for setting up RNN
params = read_yml("../params.yaml", subkey='rnn') 

In [ ]:
feats = ['Ed', 'Ew', 'solar', 'wind', 'elev', 'lon', 'lat', 'rain']
params.update({'features_list': feats})

In [ ]:
train = build_train_dict(file_paths, atm_source="HRRR", params_data = params_data,
                         features_subset = feats, spatial=True, verbose=True)

In [ ]:
# if train_create:
#     params_data.update({'hours': 1440})
#     logging.info('creating the training cases from files %s',file_paths)
#     # osp.join works on windows too, joins paths using \ or /
#     train = process_train_dict(file_paths, atm_dict = "RAWS", params_data = params_data, verbose=True)
#     train = subset_by_features(train, feats)
#     train = combine_nested(train)
# if train_write:
#     with open(train_file, 'wb') as file:
#         logging.info('Writing the rain cases into file %s',train_file)
#         pickle.dump(train, file)
# if train_read:
#     logging.info('Reading the train cases from file %s',train_file)
#     train = read_pkl(train_file)

## Train Models

In [ ]:
reproducibility.set_seed(123)

In [ ]:
params = RNNParams(params)

params.update({
    'hidden_layers': ['lstm', 'conv1d', 'dense', 'dense'],
    'hidden_units': [32, 32, 32, 16],
    'hidden_activation': ['tanh', 'tanh', 'relu', 'relu'],
    'early_stopping_patience': 25
})

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNNData

In [ ]:
rnn_dat_sp = rnn_data_wrap(train, params)
params.update({
    'loc_batch_reset': rnn_dat_sp.n_seqs, # Used to reset hidden state when location changes for a given batch
    'bmax': params_data['hours']
})

In [ ]:
# Rand Initialize
rnn0 = RNN(params)
m0, errs0, ep0 = rnn0.run_model(rnn_dat_sp, return_epochs=True)
print(f"{errs0.mean()=}")
print(f"{ep0=}")

In [ ]:
# Phys Initialize
params.update({
    'epochs':100,
    'hidden_layers': ['rnn'],
    'hidden_activation': ['relu'],
    'hidden_units': [32],
    'phys_initialize': False,
    'dropout': 0.2,
    'features_list': ["Ew", "Ed", "rain"],
    'space_fracs': [.8, .1, .1],
    'scaler': None    
})

rnn_dat1 = rnn_data_wrap(train, params)
params.update({
    'loc_batch_reset': rnn_dat_sp.n_seqs, # Used to reset hidden state when location changes for a given batch
    'bmax': params_data['hours']
})

In [ ]:
rnn1 = RNN(params)
m1, errs1, ep1 = rnn1.run_model(rnn_dat1, return_epochs=True)
print(f"{errs1.mean()=}")
print(f"{ep1=}")